In [1]:
# summaflow library
from summaflow import (
    Stats,
    GeoLayer,
    SUMMAWorkflow,
)

# 3rd party libraries
import pandas as pd
import numpy as np
import geopandas as gpd
import xarray as xr

# built-in libraries
import os

In [2]:
# paths
root_path_layers = '/Users/kasrakeshavarz/Documents/github-repos/summa-model-specific/tests/notebooks/test-stats/'

# layers' paths
landcover_path = os.path.join(root_path_layers, 'landsat-landcover')
soilclass_path = os.path.join(root_path_layers, 'usda-soil')
merithdyr_path = os.path.join(root_path_layers, 'merit-hydro')

# geolayer's path
root_path_geoms = '../../../examples/bow-at-calgary-geofabric/'
riv_path = os.path.join(root_path_geoms, 'bcalgary_rivers.shp')
cat_path = os.path.join(root_path_geoms, 'bcalgary_subbasins.shp')
hru_path = os.path.join(root_path_geoms, 'bcalgary_subbasins.shp')

____

## Building GeoLayers

In [3]:
# layers needed by the setup workflow
# elevation
elv = GeoLayer.from_maf(
    maf_stats=os.path.join(merithdyr_path, 'summaflow_stats_elv.csv'),
    maf_layer=os.path.join(merithdyr_path, 'summaflow_elv.tif'),
    maf_geolayer=os.path.join(cat_path),
    unit = 'meters',
)
# landcover
landcover = GeoLayer.from_maf(
    maf_stats=os.path.join(landcover_path, 'summaflow_stats_NA_NALCMS_landcover_2020_30m.csv'),
    maf_layer=os.path.join(landcover_path, 'summaflow_NA_NALCMS_landcover_2020_30m.tif'),
    maf_geolayer=os.path.join(cat_path),
    unit = 'dimensionless',
)
# USDA soil classes
soil = GeoLayer.from_maf(
    maf_stats=os.path.join(soilclass_path, 'summaflow_stats_soil_classes.csv'),
    maf_layer=os.path.join(soilclass_path, 'summaflow_soil_classes.tif'),
    maf_geolayer=os.path.join(cat_path),
    unit = 'dimensionless',
)

In [4]:
elv

Stats: {'mean'}
Layer: True
Geometry: True
Geolayer Unit: meter

In [5]:
landcover

Stats: {'majority', 'min', 'max', 'minority', 'variety', 'stdev', 'variance', 'q', 'count', 'median', 'mean', 'frac', 'coefficient_of_variation'}
Layer: True
Geometry: True
Geolayer Unit: dimensionless

In [6]:
soil

Stats: {'majority', 'min', 'max', 'minority', 'median', 'mean', 'frac'}
Layer: True
Geometry: True
Geolayer Unit: dimensionless

____

## Reading Geometries

In [7]:
riv_obj = gpd.read_file(riv_path)
cat_obj = gpd.read_file(cat_path)
hru_obj = gpd.read_file(hru_path)

____

In [8]:
exp = SUMMAWorkflow(
    forcing_data = ['1', '2'],
    forcing_name_mapping = {
        'pr': 'precipitation',
        'temp': 'airtemp',
    },
    forcing_unit_mapping = {
        'pr': 'mm',
        'temp': 'degC',
    },
    forcing_to_unit_mapping = {
        'pr': 'kgm^/s',
        'temp': 'K',
    },
    forcing_attrs = {
        'measurement_height': 40,
        'measurement_height_unit': 'meter',
        'forcing_time_zone': 'utc', # original timezone of the forcing datatset
        'target_time_zone': 'utc', # if UTC, SUMMA converts to local time zone internally
        'local': {
            'pr': {
                'long_name': 'precipitation at the surface',
            },
            'temp': {
                'long_name': 'Air temperature',
            },
        },
        'global': {},
    },
    topology_data = {
        'riv': riv_obj,
        'hru': hru_obj,
        'cat': cat_obj,
    },
    topology_unit_mapping = {'a': 'm', 'c': 'ha'},
    topology_to_unit_mapping = {'a': 'bar', 'c': 'dca'},
    topology_attrs = {
        'gru_fid': 'COMID',
        'hru_fid': 'COMID',
    },
    geospatial_data= {
        'elevation': elv,
        'soilTypeIndex': soil,
        'vegTypeIndex': landcover,
    },
)

____

## Manual tests on forcing files

In [10]:
ds = xr.open_dataset('../forcings/Bow_at_Banff_ERA5_remapped_domain_Bow_at_Banff_ERA5_merged_201101.nc')

In [11]:
ds

<xarray.Dataset> Size: 4MB
Dimensions:    (time: 744, hru: 163)
Coordinates:
  * time       (time) datetime64[ns] 6kB 2011-01-01 ... 2011-01-31T23:00:00
Dimensions without coordinates: hru
Data variables:
    latitude   (hru) float64 1kB ...
    longitude  (hru) float64 1kB ...
    hruId      (hru) float64 1kB ...
    airpres    (time, hru) float32 485kB ...
    LWRadAtm   (time, hru) float32 485kB ...
    SWRadAtm   (time, hru) float32 485kB ...
    pptrate    (time, hru) float32 485kB ...
    airtemp    (time, hru) float64 970kB ...
    spechum    (time, hru) float32 485kB ...
    windspd    (time, hru) float32 485kB ...
    data_step  int64 8B ...
Attributes:
    Conventions:    CF-1.6
    Author:         The data were written by SUMMA public workflow scripts
    License:        Copernicus data use license: https://cds.climate.copernic...
    History:        Created Mon Apr 14 08:46:46 2025by EASYMORE nc_remapper; ...
    easymore_hash:  ac77628b8a313cea8808b031acfc93851731005b4bb45455101f48d3a...
    Source:         Remapped by EASYMORE nc_remapper from original file: /wor...

In [41]:
ds.time.to_index().tz_localize('UTC').tz_convert('America/Edmonton').tz_localize(None)

DatetimeIndex(['2010-12-31 17:00:00', '2010-12-31 18:00:00',
               '2010-12-31 19:00:00', '2010-12-31 20:00:00',
               '2010-12-31 21:00:00', '2010-12-31 22:00:00',
               '2010-12-31 23:00:00', '2011-01-01 00:00:00',
               '2011-01-01 01:00:00', '2011-01-01 02:00:00',
               ...
               '2011-01-31 07:00:00', '2011-01-31 08:00:00',
               '2011-01-31 09:00:00', '2011-01-31 10:00:00',
               '2011-01-31 11:00:00', '2011-01-31 12:00:00',
               '2011-01-31 13:00:00', '2011-01-31 14:00:00',
               '2011-01-31 15:00:00', '2011-01-31 16:00:00'],
              dtype='datetime64[ns]', name='time', length=744, freq=None)